### **APIs with Python: Google Sheets Monitoring - Cocaine Seizures**
#### InSight Crime’s MAD Unit - (June, 2025)

##### Luis Felipe Villota Macías

---------------------



### 1. Goals

* Monitor and validate data in a shared Google Sheet using automated checks to ensure accuracy, consistency, and data quality.

* Highlight invalid or suspicious entries directly in the sheet and optionally generate weekly reports to support governance and oversight.

* Automate the process with Google Apps Script or Python, running validations on a schedule (e.g. every Friday) with minimal manual effort.




________________

### 2. Project Setup

#### 2.1 Version Control

I decided to create a single GitHub repository ([FelipeVillota/db-check-cocaine-seizures](https://github.com/FelipeVillota/db-check-cocaine-seizures)). I keep the repository `private` with the possibility to give access to the online repo at any time. 

#### 2.2 Reproducible Environment

In [ ]:
# IMPORTANT
# To create venv
# python -m venv venv-db-watch

# To activate environment, run in Terminal:
# # (optional, temporary auth) 
# Set-ExecutionPolicy -Scope Process -ExecutionPolicy Bypass 
# venv-db-watch\Scripts\activate

# Then select respective kernel --> also install ipykernel package to connect to kernel

# Update list master list
# pip freeze > requirements.txt

In [1]:
# Checking venv-db-watch works
import sys
print(sys.executable)

c:\Users\USER\Desktop\ic\db-check-cocaine-seizures\venv-db-watch\Scripts\python.exe


#### 2.3 Loading Libraries

In [ ]:
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas

In [ ]:
import os
import re
import requests
import pandas as pd
from datetime import datetime
from google.oauth2 import service_account
from googleapiclient.discovery import build
# pip freeze > requirements.txt

----------------------

### 3.  Approach

As general background and scheme, 

* I conceive an API as a portal that communicates data between different software systems, so it is key to know how to make `requests` (queries) to different `endpoints` (particular URLs as gateways of the API) to extract data and to know the level of `authentication` (permissions) required to access instances (Goodwin, 2024). Additionally, one has to consider the `status` (success or not) of the eventual `response` and the `data format` of the output (to be processed later) (Goodwin, 2024; IBM Technology, 2020).

* To interact with any API effectively, I try to get familiar with the developer's documentation (in this case also dataset codebooks). This is the guide to how the API works and how I can use its functionalities -it explains the protocols for accessing different software applications, making calls and receiving responses.

* In this case, the UCDP API is `openly available` and has a `REStful architecture` (Representational State Transfer, a client-server dynamic). This is a standard design and it means it uses `HTTP` (Hypertext Transfer Protocol) communication methods (IBM Technology, 2020). So, the `requests` (queries or petitions to the source) are made via operations under the CRUD logic (create, read, update, delete) (ibid).



* I identify that the UCDP API basics are : 

    Base URL: `https://ucdpapi.pcr.uu.se/api/`

    Endpoint structure (RESTful format): `https://ucdpapi.pcr.uu.se/api/<resource>/<version>?<pagesize=x>&<page=x>`

    Where the parameters are: 

    Target dataset: `<resource>` to be replaced with `gedevents` for the UCDP Georeferenced Event Dataset - GED

    Dataset version: `<version>` to be replaced with `24.1` which is the latest and yearly release is `24.01.24.12`

    Pagination parameters: `<pagesize=x>&<page=x>` 

    Format of requested data: `JSON` an array of objects (common notation in APIs, representing GED events)


* And, regarding rate limits (focusing on `gedevents`):

    `Requires paging:  1 to 1,000 rows per page`

    `Allows 5,000 requests/day`

    `Counters reset at midnight (UTC)`



My general idea is to create a modular client (frontend) call that extracts just the subset of data required from the API server (backend); -and, make it easily reusable for future queries.


____________________

### 4. Execution

#### 4.1 Accessing the API

In [ ]:
# Path to your downloaded JSON credentials
SERVICE_ACCOUNT_FILE = 'path/to/credentials.json'

# ID of your Google Sheet (from the URL)
SPREADSHEET_ID = 'your-spreadsheet-id'

# Range to read from your sheet (e.g. 'Sheet1!A1:Z1000')
RANGE_NAME = 'Sheet1!A1:Z1000'

# Define scopes for Google Sheets and Drive API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# Authenticate and build the service
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API to read data
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values', [])

# Convert to DataFrame for easier manipulation
df = pd.DataFrame(values[1:], columns=values[0])
print(df.head()) 

#### 4.2 Data Analysis

In [ ]:
# 1. What are the five deadliest municipalities (adm_2) overall in 2024?

# Focus on variable:
# best (integer): "The best (most likely) estimate of total fatalities resulting from an event." (Högbladh, 2024; Sundberg et. al, 2013) 

full_2024.groupby('adm_2')['best'].sum().nlargest(5)

adm_2
Pokrovsk raion                     17849
Deir al-Balah governorate           3929
Bakhmut raion                       3603
Gaza governorate                    3386
Gaza ash Shamaliyah governorate     2851
Name: best, dtype: int64

In [ ]:
# 2. What are the five deadliest municipalities (adm_2) just for civilians in 2024?

# Focus on variable:
# deaths_civilians (integer):"The best estimate of dead civilians in the event.  For non-state or state-based events, this is the number of collateral damage resulting in fighting between side a and side integer b. For one-sided violence, it is the number of civilians killed by side a." (Högbladh, 2024; Sundberg et. al, 2013) 

full_2024.groupby('adm_2')['deaths_civilians'].sum().nlargest(5)

adm_2
Deir al-Balah governorate          1801
Gaza governorate                   1234
Gaza ash Shamaliyah governorate     871
El Fasher district                  797
Rafah governorate                   729
Name: deaths_civilians, dtype: int64

In [ ]:
# For Q3 & Q4: Given the instructions and the datasets consulted, I interpret different possibilities to answer them,


# Option 1 -> I can only find the highest and lowest count of unique events in DEC 2023 (strictly using just the official latest dataset and the defined date filter)

# 3. Which province (adm_1) has seen the largest increase in overall violence since December 2023?
# id (integer) = "A unique numeric ID identifying each event." (Högbladh, 2024; Sundberg et. al, 2013) 
print("OPTION 1: The province with the largest increase in overall violence (in number of unique events) since December 2023 is:")
print(since_dec_2023.groupby('adm_1')['id'].count().nlargest(1))

#4. Which province (adm_1) has seen the largest decrease in overall violence since December 2023?
print("OPTION 1: The province with the largest decrease in overall violence (in number of unique events) since December 2023 is:")
since_dec_2023.groupby('adm_1')['id'].count().nsmallest(1) # Of course, there must be at least 1 event (lowest count), several have just 1 event 


OPTION 1: The province with the largest increase in overall violence (in number of unique events) since December 2023 is:
adm_1
Gaza Strip    65
Name: id, dtype: int64
OPTION 1: The province with the largest decrease in overall violence (in number of unique events) since December 2023 is:


adm_1
Adamawa state    1
Name: id, dtype: int64

In [ ]:
# Option 2 -> using merged df and comparing

combined_dfs['date_end'] = pd.to_datetime(combined_dfs['date_end']) # just to confirm the data type

# Date splits
before = combined_dfs[combined_dfs['date_end'] < '2023-12-01'] # since DEC 2023
after = combined_dfs[combined_dfs['date_end'] >= '2023-12-01'] # onwards

change = (
    after['adm_1'].value_counts() - 
    before['adm_1'].value_counts()).fillna(0) # comparing province series of unique events, gives positive and negative changes

print(f"OPTION 2: Largest increase since DEC 2023 (province level): {change.idxmax()} ({int(change.max())} more events)") # printing index label for largest value (province) and actual count (largest)
print(f"OPTION 2: Largest decrease since DEC 2023 (province level): {change.idxmin()} ({int(change.min())} fewer events)") # printing index label for lowest value (province) and actual count (min)

OPTION 2: Largest increase since DEC 2023 (province level): Kursk oblast (498 more events)
OPTION 2: Largest decrease since DEC 2023 (province level): Rif Dimashq governorate (-17401 fewer events)


### References

Davies, Shawn, Garoun Engström, Therese Pettersson & Magnus Öberg (2024). Organized violence 1989-2023, and the prevalence of organized crime groups. Journal of Peace Research 61(4).

Goodwin, M. (2024, April 9). What Is an API (Application Programming Interface)? IBM. https://www.ibm.com/think/topics/api

Högbladh, Stina. (2024). “UCDP GED Codebook version 24.1”, Department of Peace and Conflict Research, Uppsala University

IBM Technology (Director). (2020, October 23). What is a REST API? [Video recording]. https://www.youtube.com/watch?v=lsMQRaeKNDk

JSON. (n.d.). Retrieved April 7, 2025, from https://www.json.org/json-en.html

Sundberg, Ralph and Erik Melander (2013) Introducing the UCDP Georeferenced Event Dataset. Journal of Peace Research 50(4). 523-532

UCDP Application Programming Interface (API). (n.d.). Retrieved April 7, 2025, from https://ucdp.uu.se/apidocs/


________________